In [16]:
import math
import numpy as np
import TasmanianSG
import numpy as np
import math
from random import uniform
from datetime import datetime

In [17]:
def oscillatory(xvec, w, cvec):
    s = math.pi * 2 * w
    d = len(xvec)
    for i in range(d):
        s += cvec[i] * xvec[i]
    return math.cos(s)

def product_peak(xvec, wvec, cvec):
    p = 1
    d = len(xvec)
    for i in range(d):
        p *= (cvec[i] ** -2 + (xvec[i] - wvec[i]) ** 2) ** -1
    return p

def corner_peak(xvec, cvec):
    s = 1
    d = len(xvec)
    for i in range(d):
        s += cvec[i] * xvec[i]
    return s ** -(d + 1)

def gaussian(xvec, wvec, cvec, t):
    s = 0
    d = len(xvec)
    for i in range(d):
        s += (cvec[i] ** 2) * t * (xvec[i] - wvec[i]) ** 2
    return math.exp(-s)

def continuous(xvec, wvec, cvec):
    s = 0
    d = len(xvec)
    for i in range(d):
        s += cvec[i] * abs(xvec[i] - wvec[i])
    return math.exp(-s)

def discontinous(xvec, wvec, cvec):
    s = 0
    d = len(xvec)
    for i in range(d):
        if i in [0, 1]:
            if xvec[i] > wvec[i]:
                return 0
        s += cvec[i] * xvec[i]
    return math.exp(s)

In [18]:
def fun(f, iDepth=5, method="normal" ,fTol = 1.E-5, refinement_level=5):
    grid  = TasmanianSG.TasmanianSparseGrid()
    grid2 = TasmanianSG.TasmanianSparseGrid()
    
    # params
    iDim = 1
    iOut = 1
    which_basis = 1

    # xvec = aPnts
    aPnts = np.empty([1000, ])  
    for iI in range(1000):
        aPnts[iI] = uniform(-1.0, 1.0)

    # Result
    aTres = np.empty([1000,])
    for iI in range(1000):
        aTres[iI] = f(aPnts[iI])

    # construct sparse grid
    grid.makeLocalPolynomialGrid(iDim, iOut, iDepth, which_basis, "localp")
    aPoints = grid.getPoints()
    iNumP1 = aPoints.shape[0]
    aVals = np.empty([aPoints.shape[0], 1])
    for iI in range(aPoints.shape[0]):
        aVals[iI] = f(aPoints[iI])
    grid.loadNeededPoints(aVals)


    if method == "normal":
        aRes = grid.evaluateBatch(aPnts)
    elif method == "adaptive":
        fError = []
        numpts = []
        for iK in range(refinement_level):
            grid.setSurplusRefinement(fTol, 1, "fds")   #also use fds, or other rules
            aPoints = grid.getNeededPoints()
            aVals = np.empty([aPoints.shape[0], 1])
            for iI in range(aPoints.shape[0]):
                aVals[iI] = f(aPoints[iI])
            grid.loadNeededPoints(aVals)
            aRes = grid.evaluateBatch(aPnts)
            fError.append(max(np.fabs(aRes[:,0] - aTres)))
            numpts.append(grid.getNumPoints())
        plt.plot(fError, numpts)